In [2]:
import obspy
from obspy import read, Stream, UTCDateTime
import glob
import os
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot
import matplotlib
import time


# Loading all data

In [4]:
#This takes a while - around 30 minutes!!

node_streams = {}
no_of_nodes = 47

start_time = time.time()


for filepath in glob.glob("GA_refraction_test/GA_refraction_test/GA-refraction-survey-labelled/*.E.segy")[:no_of_nodes]:     #up to 47 doesn't inlcude our failsafe node
    filename = os.path.basename(filepath) 
    node_id = filename[:7]
    stream = read(filepath, format = "SEGY")
    
    for i, trace in enumerate(stream):
        trace.stats.starttime += i * datetime.timedelta(seconds = 6) # NOT sure that the traces are automatically ordered.
    stream.merge()
    
    node_streams[node_id] = stream

end_time = time.time()
elapsed_time = end_time - start_time
print(f"The task took {elapsed_time:.2f} seconds.")

The task took 637.79 seconds.


In [5]:
file_path = "TEST_symaug25A.txt"
coords = pd.read_csv(file_path)
coords = pd.read_csv("TEST_symaug25A.txt", header=None, usecols=[0, 1, 2])
coords.columns = ['ID', 'Northing', 'Easting']
nodes_coords = coords.iloc[:no_of_nodes]
nodes_coords['ID'] = nodes_coords['ID'].str.extract(r'N(\d+)')[0].astype(int).apply(lambda x: f'node_{x:02d}')


shots_coords = coords.iloc[no_of_nodes+1:]

C:\Users\u06008\AppData\Local\Temp\ipykernel_18828\4156282284.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_coords['ID'] = nodes_coords['ID'].str.extract(r'N(\d+)')[0].astype(int).apply(lambda x: f'node_{x:02d}')


In [6]:
for node_id, stream in node_streams.items():
    stream.merge()
    northing = nodes_coords.loc[nodes_coords["ID"] == node_id, "Northing"].values[0]
    easting = nodes_coords.loc[nodes_coords["ID"] == node_id, "Easting"].values[0]
    for tr in stream:    
        tr.stats.coordinates = {"x": easting, "y": northing}    

## Loading shot times and information

In [7]:
# Read in shot gathers
shot_log = pd.read_csv("shot_information.txt")
shot_log.head(5)

shot_log_ordered = (shot_log.sort_values(by = ["POINT", "PINDEX"]))

shot_log_ordered_and_filtered = shot_log_ordered[shot_log_ordered["PINDEX"]<=20]

In [8]:
shot_points = np.arange(9, 70, 4) # shot point 9 is the first (it is normally 1 but I omitted these). 

for shot in shot_points:
    point_df = shot_log_ordered_and_filtered[shot_log_ordered_and_filtered["POINT"]==shot]
    point_df_hammer = point_df[(point_df["PINDEX"] >=1) & (point_df["PINDEX"] <11)]
    point_df_peg = point_df[(point_df["PINDEX"] >=11) & (point_df["PINDEX"] <=20)]
    output_folder = f"Spoint_{shot:02d}"
    os.makedirs(output_folder, exist_ok=True)
    for i, datetime in enumerate(np.array(point_df_peg['UTC']), start=1):
        shot_time = UTCDateTime(datetime)
        window_start = shot_time
        window_end = shot_time + 0.6
        stream = obspy.Stream([s[0] for s in node_streams.values()])
        sliced = stream.slice(starttime=window_start, endtime=window_end)
        output_path = os.path.join(output_folder, f"point_{shot:02d}_peg_hit_{i:02d}.segy")
        sliced.write(output_path, format="SEGY")

    for i, datetime in enumerate(np.array(point_df_hammer['UTC']), start=1):
        shot_time = UTCDateTime(datetime)
        window_start = shot_time
        window_end = shot_time + 0.6    # length of time (ms) that is exported from smartsolo products
        stream = obspy.Stream([s[0] for s in node_streams.values()])
        sliced = stream.slice(starttime=window_start, endtime=window_end)
        output_path = os.path.join(output_folder, f"point_{shot:02d}_hammer_hit_{i:02d}.segy")
        sliced.write(output_path, format="SEGY")
    

In [9]:
# Create output folders
os.makedirs("S_stacked_shots/peg", exist_ok=True)
os.makedirs("S_stacked_shots/hammer", exist_ok=True)

In [10]:
# Loop through all shot folders
for shot_folder in sorted(glob.glob("Spoint_*")):
    shot_id = shot_folder.split("_")[1]  

    # Stack PEG hits
    peg_files = sorted(glob.glob(f"{shot_folder}/point_{shot_id}_peg_hit_*.segy"))
    if peg_files:
        peg_streams = [read(fp, format="SEGY") for fp in peg_files]
        stacked_peg = Stream()
        for i in range(len(peg_streams[0])):
            traces = [st[i] for st in peg_streams]
            stacked_data = np.mean([tr.data for tr in traces], axis=0)
            stacked_trace = traces[0].copy()
            stacked_trace.data = stacked_data
            stacked_peg.append(stacked_trace)
        stacked_peg.write(f"S_stacked_shots/peg/stacked_peg_point_{shot_id}.sgy", format="SEGY")   # Refrapy only registers segy files with the '.sgy' format

    # Stack HAMMER hits
    hammer_files = sorted(glob.glob(f"{shot_folder}/point_{shot_id}_hammer_hit_*.segy"))
    if hammer_files:
        hammer_streams = [read(fp, format="SEGY") for fp in hammer_files]
        stacked_hammer = Stream()
        for i in range(len(hammer_streams[0])):
            traces = [st[i] for st in hammer_streams]
            stacked_data = np.mean([tr.data for tr in traces], axis=0)
            stacked_trace = traces[0].copy()
            stacked_trace.data = stacked_data
            stacked_hammer.append(stacked_trace)
        stacked_hammer.write(f"S_stacked_shots/hammer/stacked_hammer_point_{shot_id}.sgy", format="SEGY")